In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('dataset.csv',delimiter=";")
template = pd.read_csv("template.csv",delimiter=";")
data.columns

Index(['uuid', 'default', 'account_amount_added_12_24m',
       'account_days_in_dc_12_24m', 'account_days_in_rem_12_24m',
       'account_days_in_term_12_24m', 'account_incoming_debt_vs_paid_0_24m',
       'account_status', 'account_worst_status_0_3m',
       'account_worst_status_12_24m', 'account_worst_status_3_6m',
       'account_worst_status_6_12m', 'age', 'avg_payment_span_0_12m',
       'avg_payment_span_0_3m', 'merchant_category', 'merchant_group',
       'has_paid', 'max_paid_inv_0_12m', 'max_paid_inv_0_24m', 'name_in_email',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'status_last_archived_0_24m',
       'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m',
       'status_max_archived_0_6_months', 'status_max_archived_0_12_months',

In [3]:
(data.isna().sum()/len(data)).sort_values(ascending=False).head(10)

worst_status_active_inv                0.695317
account_worst_status_12_24m            0.667770
account_worst_status_6_12m             0.603645
account_incoming_debt_vs_paid_0_24m    0.593292
account_worst_status_3_6m              0.577159
account_status                         0.543861
account_worst_status_0_3m              0.543861
avg_payment_span_0_3m                  0.493168
avg_payment_span_0_12m                 0.238417
num_active_div_by_paid_inv_0_12m       0.229445
dtype: float64

In [4]:
data.drop(columns=["worst_status_active_inv"
                   ,"account_worst_status_12_24m"
                   ,"account_worst_status_6_12m"
                   ,"account_incoming_debt_vs_paid_0_24m"
                   ,"account_worst_status_3_6m"
                   ,"account_status"
                   ,"account_worst_status_0_3m"
                   ,"avg_payment_span_0_3m"
                   # And uuid
                   ,"uuid"
                   
                   ]
          ,inplace=True)


In [5]:
# Missing values
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy="most_frequent")

# Tcheck that there is no na in our y
print(data.loc[:,"default"].isna().sum())
data.dropna(subset=["default"],inplace=True)

X = data.copy()
y = X.pop("default")


10000


In [6]:
# Scaling 
from sklearn.preprocessing import StandardScaler
cat_cols = [
            #,"account_status"
            #,"account_worst_status_0_3m"
            #,"account_worst_status_12_24m"
            #,"account_worst_status_3_6m"
            #,"account_worst_status_6_12m"
            "merchant_category"
            ,"merchant_group"
            ,"status_last_archived_0_24m"
            ,"status_2nd_last_archived_0_24m"
            ,"status_3rd_last_archived_0_24m"
            ,"status_max_archived_0_6_months"
            ,"status_max_archived_0_12_months"
            ,"status_max_archived_0_24_months"
            #,"worst_status_active_inv"
            ,"name_in_email"
            ,"has_paid"
            ]
X_num = X.drop(columns=cat_cols).astype("float64")
X_cat = X.loc[:,cat_cols].astype("category")

# No need to impute X_cat we have no missing values
print("Missing value in X_cat :",X_cat.isna().sum().sum())

X_num.columns


Missing value in X_cat : 0


Index(['account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m', 'age',
       'avg_payment_span_0_12m', 'max_paid_inv_0_12m', 'max_paid_inv_0_24m',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'recovery_debt', 'sum_capital_paid_account_0_12m',
       'sum_capital_paid_account_12_24m', 'sum_paid_inv_0_12m', 'time_hours'],
      dtype='object')

In [7]:
num_impute = SimpleImputer(strategy="mean")
X_num = pd.DataFrame(num_impute.fit_transform(X_num),columns=X_num.columns)
print("Missinng values in X_num :",X_num.isna().sum().sum())

Missinng values in X_num : 0


In [8]:
# One hot encoding
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False,handle_unknown="ignore")
X_cat = pd.DataFrame(ohe.fit_transform(X_cat),columns=ohe.get_feature_names_out(X_cat.columns))
X_cat.head()

,merchant_category_Adult Shoes & Clothing,merchant_category_Automotive Parts & Accessories,merchant_category_Bags & Wallets,merchant_category_Body & Hair Care,merchant_category_Books & Magazines,merchant_category_Car electronics,merchant_category_Children Clothes & Nurturing products,merchant_category_Children toys,merchant_category_Cleaning & Sanitary,merchant_category_Collectibles,...,name_in_email_F,name_in_email_F+L,name_in_email_F1+L,name_in_email_Initials,name_in_email_L,name_in_email_L1+F,name_in_email_Nick,name_in_email_no_match,has_paid_False,has_paid_True
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression


num_cols = ['account_amount_added_12_24m', 'account_days_in_dc_12_24m',
       'account_days_in_rem_12_24m', 'account_days_in_term_12_24m', 'age',
       'avg_payment_span_0_12m', 'max_paid_inv_0_12m', 'max_paid_inv_0_24m',
       'num_active_div_by_paid_inv_0_12m', 'num_active_inv',
       'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m',
       'num_arch_ok_12_24m', 'num_arch_rem_0_12m',
       'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m',
       'num_unpaid_bills', 'recovery_debt', 'sum_capital_paid_account_0_12m',
       'sum_capital_paid_account_12_24m', 'sum_paid_inv_0_12m', 'time_hours']

cat_cols = [
            "merchant_category" ,"merchant_group"
            ,"status_last_archived_0_24m","status_2nd_last_archived_0_24m"
            ,"status_3rd_last_archived_0_24m","status_max_archived_0_6_months"
            ,"status_max_archived_0_12_months","status_max_archived_0_24_months"
            ,"name_in_email","has_paid"
            ]

num_pipe = Pipeline([("Imputer",SimpleImputer(strategy="mean"))
                     ,("Scaler",StandardScaler())
                    ])
cat_pipe = Pipeline([
                    ("Encoder",OneHotEncoder(sparse=False,handle_unknown="ignore"))
                     ])

preproc_pipe = make_column_transformer(
                 (num_pipe,num_cols)
                ,(cat_pipe,cat_cols)
                ,remainder="drop"
                )


pipe = Pipeline([("preproc",preproc_pipe)
                 ,("model",LogisticRegression(max_iter=10_000))
                 ])

pipe.fit(X,y)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaler',
                                                                   StandardScaler())]),
                                                  ['account_amount_added_12_24m',
                                                   'account_days_in_dc_12_24m',
                                                   'account_days_in_rem_12_24m',
                                                   'account_days_in_term_12_24m',
                                                   'age',
                                                   'avg_payment_span_0_12m',
                                                   'max_paid_inv_0_12m',
                                                   'max_paid_inv_0_24m',
                                                   'num_active_...
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['merchant_category',
                                                   'merchant_group',
                                                   'status_last_archived_0_24m',
                                                   'status_2nd_last_archived_0_24m',
                                                   'status_3rd_last_archived_0_24m',
                                                   'status_max_archived_0_6_months',
                                                   'status_max_archived_0_12_months',
                                                   'status_max_archived_0_24_months',
                                                   'name_in_email',
                                                   'has_paid'])])),
                ('model', LogisticRegression(max_iter=10000))])

In [10]:
# Saving the trained pipe in the cloud folder
import joblib
joblib.dump(pipe,"cloud/pipe.joblib")

['cloud/pipe.joblib']

In [12]:
# Loading the pipe to make sure it works
new_pipe = joblib.load("cloud/pipe.joblib")

In [ ]:
line= X.iloc[0:1,:]
line

,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,age,avg_payment_span_0_12m,merchant_category,merchant_group,has_paid,max_paid_inv_0_12m,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
0,0,0.0,0.0,0.0,20,12.692308,Dietary supplements,Health & Beauty,True,31638.0,...,1,1,1,1,1,0,0,0,178839,9.653333


In [ ]:
int(pipe.predict(line)[0])

0